In [1]:
import os
os.chdir('../src')

### load getPretrained module

In [2]:
%run modules/model/getPretrained.py

tf.__version__ : 1.13.1
keras.__version__: 2.2.4-tf


In [3]:
ModelFileDict = getModelFileDict()
ModelFileDict

{'Xception': '../models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5',
 'VGG16': '../models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
 'VGG16_includeTop': '../models/vgg16_weights_tf_dim_ordering_tf_kernels.h5',
 'VGG19': '../models/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
 'InceptionV3': '../models/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5',
 'MobileNet': '../models/mobilenet_1_0_224_tf_no_top.h5',
 'DenseNet': '../models/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5',
 'NASNet': '../models/nasnet_mobile_no_top.h5',
 'ResNet50': '../models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
 'InceptionResNetV2': '../models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'}

In [4]:
@lD.log(logBase + '.model')
def modelImageNet(logger, modelName, weightsFile=None, input_shape=(224, 224, 3)):

    try:

        if weightsFile is not None:
            weights = weightsFile
        else:
            weights = 'imagenet'

        if   modelName == 'Xception':
            base_model  = keras.applications.xception.Xception(input_shape=input_shape, include_top=False, weights=weights)
        elif modelName == 'VGG16':
            base_model  = keras.applications.vgg16.VGG16(input_shape=input_shape, include_top=False, weights=weights)
        elif modelName == 'VGG16_includeTop':
            base_model  = keras.applications.vgg16.VGG16(input_shape=input_shape, include_top=True, weights=weights)
        elif modelName == 'VGG19':
            base_model  = keras.applications.vgg19.VGG19(input_shape=input_shape, include_top=False, weights=weights)
        elif modelName == 'ResNet50':
            base_model  = keras.applications.resnet50.ResNet50(input_shape=input_shape, include_top=False, weights=weights)
        elif modelName == 'InceptionV3':
            base_model  = keras.applications.inception_v3.InceptionV3(input_shape=input_shape, include_top=False, weights=weights)
        elif modelName == 'InceptionResNetV2':
            base_model  = keras.applications.inception_resnet_v2.InceptionResNetV2(input_shape=input_shape, include_top=False, weights=weights)
        elif modelName == 'MobileNet':
            base_model  = keras.applications.mobilenet.MobileNet(input_shape=input_shape, include_top=False, weights=weights)
        elif modelName == 'DenseNet':
            base_model  = keras.applications.densenet.DenseNet121(input_shape=input_shape, include_top=False, weights=weights)
        elif modelName == 'NASNet':
            base_model  = keras.applications.nasnet.NASNetMobile(input_shape=input_shape, include_top=False, weights=weights)

        return base_model

    except Exception as e:

        logger.error('Unable to get model: {} \n{}'.format(modelName, str(e)))

In [5]:
modelwTop = modelImageNet('VGG16_includeTop', ModelFileDict['VGG16_includeTop'])
modelwTop.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
bloc

In [6]:
@lD.log(logBase + '.outputTensorBoard')
def outputTensorBoard(logger, subfolder=None):

    try:

        tfboardFolder = '../notebooks/tensorlog/'

        if subfolder is not None:
            tfboardFolder = os.path.join(tfboardFolder, subfolder)

        if os.path.exists(tfboardFolder):
            shutil.rmtree(tfboardFolder)

        os.makedirs(tfboardFolder)

        with tf.Session() as sess:
            tfWriter = tf.summary.FileWriter(tfboardFolder, sess.graph)
            tfWriter.close()

    except Exception as e:

        logger.error('Unable to output tensorboard \n{}'.format(str(e)))

In [7]:
outputTensorBoard('VGG16_includeTop_folder')

In [8]:
modelwTop.layers

In [10]:
outputFromMaxPooling2D = modelwTop.layers[-5].output
outputFromMaxPooling2D

<tf.Tensor 'block5_pool/MaxPool:0' shape=(?, 7, 7, 512) dtype=float32>

In [11]:
print('flatten layer', modelwTop.layers[-4].output)
print('dense layer',   modelwTop.layers[-3].output)

flatten layer Tensor("flatten/Reshape:0", shape=(?, 25088), dtype=float32)
dense layer Tensor("fc1/Relu:0", shape=(?, 4096), dtype=float32)


In [12]:
with tf.variable_scope('additionalLayer'):
    flattenLayer  = tf.layers.Flatten(data_format='channels_last', name='flattenLayer')
    flattenOutput = flattenLayer(outputFromMaxPooling2D)

In [13]:
outputTensorBoard('VGG16_changed')

In [14]:
with tf.variable_scope('anotherLayer'):
    denseLayer = tf.layers.Dense(units=100, activation=tf.nn.relu, name='denseLayer')
    output     = denseLayer(flattenOutput)

In [15]:
outputTensorBoard('VGG16_changed_v2')

After changing layers, summary remain as the original

In [ ]:
modelwTop.summary()

In [ ]:
modelwTop.input

In [ ]:
output

In [ ]:
modelNew = keras.models.Model(inputs=modelwTop.input, outputs=output)

In [ ]:
modelNew.summary()